<a href="https://colab.research.google.com/github/pranavmeraga/pmeraga/blob/main/LSTM%2BCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization, Dropout,
                                   LSTM, Dense, TimeDistributed, Flatten, Input,
                                   GlobalAveragePooling2D, Reshape, Concatenate)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
import os
import warnings
import re
warnings.filterwarnings('ignore')

class CNNLSTMKeyholeForecaster:
    def __init__(self, sequence_length=20, forecast_horizon=5,
                 img_height=128, img_width=128, channels=1,
                 target_features=['melt_pool_width', 'melt_pool_depth', 'keyhole_width', 'keyhole_depth', 'porosity'],
                 include_power_feature=True):
        """
        Initialize the CNN-LSTM forecaster for keyhole dimensions from X-ray images

        Parameters:
        - sequence_length: Number of previous frames to use for prediction
        - forecast_horizon: Number of future timesteps to predict
        - img_height, img_width: Image dimensions (will be resized to this)
        - channels: Number of image channels (1 for grayscale X-ray)
        - target_features: Features to predict from images
        - include_power_feature: Whether to include laser power as an additional input feature
        """
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon
        self.img_height = img_height
        self.img_width = img_width
        self.channels = channels
        self.target_features = target_features
        self.n_features = len(target_features)
        self.include_power_feature = include_power_feature

        # Scalers
        self.target_scaler = MinMaxScaler()
        self.power_scaler = MinMaxScaler()  # For laser power normalization
        self.image_scaler = 1.0 / 255.0  # Normalize images to [0,1]

        self.model = None
        self.history = None
        self.X_test = None
        self.y_test = None
        self.power_test = None

    def extract_power_from_filename(self, filename):
        """
        Extract laser power from filename

        Expected format: 017_Al6061_P200W_t4ms_U18G16_spot0255_original.jpg
        Returns: Power value (e.g., 200 for P200W)
        """
        try:
            # Use regex to find P followed by digits and W
            power_match = re.search(r'P(\d+)W', filename)
            if power_match:
                return int(power_match.group(1))
            else:
                # Try alternative pattern if main one fails
                alt_power_match = re.search(r'P(\d+)_', filename)
                if alt_power_match:
                    return int(alt_power_match.group(1))
                else:
                    print(f"Warning: Could not extract power from filename: {filename}")
                    return 200  # Default fallback
        except Exception as e:
            print(f"Error extracting power from {filename}: {e}")
            return 200

    def extract_spot_index_from_filename(self, filename):
        """
        Extract spot index from filename

        Expected format: 017_Al6061_P200W_t4ms_U18G16_spot0255_original.jpg
        Returns: Spot index (e.g., 255 for spot0255)
        """
        try:
            # Use regex to find spot followed by digits
            spot_match = re.search(r'spot(\d+)', filename)
            if spot_match:
                return int(spot_match.group(1))
            else:
                print(f"Warning: Could not extract spot index from filename: {filename}")
                return 0  # Default fallback
        except Exception as e:
            print(f"Error extracting spot index from {filename}: {e}")
            return 0

    def load_trial_sequences(self, trial, trial_data, image_col='image_path'):
        """
        Load all image sequences and labels for a single trial.

        Parameters:
        - trial: trial identifier
        - trial_data: dataframe containing image paths and target labels for this trial
        - image_col: column name containing image paths

        Returns: X, y, power_sequences (sequences and future predictions)
        """
        # Sort by spot index to ensure temporal order
        if 'spot_index' in trial_data.columns:
            trial_data = trial_data.sort_values('spot_index')
        elif 'abs_index' in trial_data.columns:
            trial_data = trial_data.sort_values('abs_index')
        else:
            trial_data = trial_data.sort_values(image_col)

        images = self.load_and_preprocess_images(trial_data[image_col].values)

        # Ensure target features are numeric before scaling
        targets_df = trial_data[self.target_features].apply(pd.to_numeric, errors='coerce')
        targets = self.target_scaler.transform(targets_df.values)

        if self.include_power_feature:
            # Ensure power values are numeric before scaling
            power_values = pd.to_numeric(trial_data['laser_power'], errors='coerce').values
            power_normalized = self.power_scaler.transform(power_values.reshape(-1, 1)).flatten()
            return self.create_sequences_with_power(images, targets, power_normalized)
        else:
            return self.create_sequences(images, targets)

    def load_and_preprocess_images(self, image_paths):
        """Load and preprocess images from file paths"""
        images = []

        for path in image_paths:
            if not os.path.exists(path):
                print(f"Warning: Image not found: {path}")
                img = np.zeros((self.img_height, self.img_width), dtype=np.float32)
            else:
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Warning: Could not load image: {path}")
                    img = np.zeros((self.img_height, self.img_width), dtype=np.float32)
                else:
                    img = cv2.resize(img, (self.img_width, self.img_height))
                    img = img.astype(np.float32) * self.image_scaler

            img = np.expand_dims(img, axis=-1)  # Add channel dimension
            images.append(img)

        return np.array(images)

    def create_sequences(self, images, targets):
        """
        Create sequences for CNN-LSTM training (without power feature)

        Parameters:
        - images: Array of preprocessed images
        - targets: Array of target values (keyhole dimensions)

        Returns:
        - X: Input image sequences
        - y: Target sequences
        """
        X, y = [], []

        for i in range(len(images) - self.sequence_length - self.forecast_horizon + 1):
            # Input sequence of images
            X.append(images[i:(i + self.sequence_length)])

            # Target sequence (multi-step ahead predictions)
            y.append(targets[(i + self.sequence_length):(i + self.sequence_length + self.forecast_horizon)])

        return np.array(X), np.array(y)

    def create_sequences_with_power(self, images, targets, power_values):
        """
        Create sequences for CNN-LSTM training with power feature

        Parameters:
        - images: Array of preprocessed images
        - targets: Array of target values (keyhole dimensions)
        - power_values: Array of normalized power values

        Returns:
        - X: Input image sequences
        - y: Target sequences
        - power_seq: Power value sequences
        """
        X, y, power_seq = [], [], []

        for i in range(len(images) - self.sequence_length - self.forecast_horizon + 1):
            # Input sequence of images
            X.append(images[i:(i + self.sequence_length)])

            # Target sequence (multi-step ahead predictions)
            y.append(targets[(i + self.sequence_length):(i + self.sequence_length + self.forecast_horizon)])

            # Power sequence (current implementation uses the power at the start of sequence)
            # You can modify this to use power at prediction time or average power
            power_seq.append(power_values[i + self.sequence_length - 1])  # Power at end of input sequence

        return np.array(X), np.array(y), np.array(power_seq)

    def build_cnn_feature_extractor(self):
        """Build CNN feature extractor for individual frames"""
        cnn_input = Input(shape=(self.img_height, self.img_width, self.channels))

        x = Conv2D(32, 3, activation='relu', padding='same')(cnn_input)
        x = BatchNormalization()(x)
        x = MaxPooling2D()(x)

        x = Conv2D(64, 3, activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D()(x)

        x = Conv2D(128, 3, activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = GlobalAveragePooling2D()(x)

        features = Dense(128, activation='relu', name='image_features')(x)
        features = Dropout(0.3)(features)

        return Model(inputs=cnn_input, outputs=features)

    def build_model(self):
        """Build the complete CNN-LSTM model with optional power input"""
        # Image input
        image_input = Input(shape=(self.sequence_length, self.img_height, self.img_width, self.channels),
                           name='image_input')

        # CNN feature extractor (applied to each frame)
        cnn_model = self.build_cnn_feature_extractor()

        # Apply CNN to each frame in the sequence
        cnn_features = TimeDistributed(cnn_model)(image_input)

        if self.include_power_feature:
            # Power input (scalar value)
            power_input = Input(shape=(1,), name='power_input')

            # Repeat power value for each timestep in sequence
            power_repeated = tf.keras.layers.RepeatVector(self.sequence_length)(power_input)

            # Concatenate CNN features with power
            combined_features = Concatenate(axis=-1)([cnn_features, power_repeated])

            # LSTM layers for temporal modeling
            lstm_out = LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(combined_features)
            lstm_out = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(lstm_out)
            lstm_out = LSTM(32, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)(lstm_out)

            # Include power in final layers
            power_dense = Dense(16, activation='relu')(power_input)
            combined_final = Concatenate()([lstm_out, power_dense])

            # Dense layers for prediction
            x = Dense(64, activation='relu')(combined_final)
            x = Dropout(0.3)(x)
            x = Dense(32, activation='relu')(x)
            x = Dropout(0.2)(x)

            # Output layer
            output = Dense(self.forecast_horizon * self.n_features, activation='linear')(x)
            output = Reshape((self.forecast_horizon, self.n_features))(output)

            model = Model(inputs=[image_input, power_input], outputs=output)

        else:
            # LSTM layers for temporal modeling (without power)
            lstm_out = LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(cnn_features)
            lstm_out = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(lstm_out)
            lstm_out = LSTM(32, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)(lstm_out)

            # Dense layers for prediction
            x = Dense(64, activation='relu')(lstm_out)
            x = Dropout(0.3)(x)
            x = Dense(32, activation='relu')(x)
            x = Dropout(0.2)(x)

            # Output layer
            output = Dense(self.forecast_horizon * self.n_features, activation='linear')(x)
            output = Reshape((self.forecast_horizon, self.n_features))(output)

            model = Model(inputs=image_input, outputs=output)

        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )

        return model

    def prepare_data_by_trial(self, df_combined, train_trials, val_trials, test_trials):
        """
        Prepare training/validation/test data by trial splitting

        Parameters:
        - df_combined: Combined dataframe with all trials
        - train_trials, val_trials, test_trials: Lists of trial names for each split

        Returns:
        - Tuple of (train_data, val_data, test_data)
        """
        X_train, y_train, power_train = [], [], []
        X_val, y_val, power_val = [], [], []
        X_test, y_test, power_test = [], [], []

        for trial in df_combined['trial'].unique():
            trial_data = df_combined[df_combined['trial'] == trial]

            print(f"Processing trial {trial}: {len(trial_data)} samples")

            # Load trial sequences
            if self.include_power_feature:
                trial_X, trial_y, trial_power = self.load_trial_sequences(trial, trial_data, image_col='image_path')
            else:
                trial_X, trial_y = self.load_trial_sequences(trial, trial_data, image_col='image_path')
                trial_power = None

            if len(trial_X) == 0:
                print(f"Warning: No sequences generated for trial {trial}")
                continue

            print(f"  Generated {len(trial_X)} sequences")

            # Assign to appropriate split
            if trial in train_trials:
                X_train.append(trial_X)
                y_train.append(trial_y)
                if self.include_power_feature:
                    power_train.append(trial_power)
            elif trial in val_trials:
                X_val.append(trial_X)
                y_val.append(trial_y)
                if self.include_power_feature:
                    power_val.append(trial_power)
            elif trial in test_trials:
                X_test.append(trial_X)
                y_test.append(trial_y)
                if self.include_power_feature:
                    power_test.append(trial_power)

        # Concatenate arrays
        X_train = np.concatenate(X_train) if X_train else np.array([])
        y_train = np.concatenate(y_train) if y_train else np.array([])
        X_val = np.concatenate(X_val) if X_val else np.array([])
        y_val = np.concatenate(y_val) if y_val else np.array([])
        X_test = np.concatenate(X_test) if X_test else np.array([])
        y_test = np.concatenate(y_test) if y_test else np.array([])

        if self.include_power_feature:
            power_train = np.concatenate(power_train) if power_train else np.array([])
            power_val = np.concatenate(power_val) if power_val else np.array([])
            power_test = np.concatenate(power_test) if power_test else np.array([])
        else:
            power_train = power_val = power_test = None

        print(f"\nFinal data shapes:")
        print(f"  Train: X={X_train.shape}, y={y_train.shape}", end="")
        if self.include_power_feature:
            print(f", power={power_train.shape}")
        else:
            print()
        print(f"  Val:   X={X_val.shape}, y={y_val.shape}", end="")
        if self.include_power_feature:
            print(f", power={power_val.shape}")
        else:
            print()
        print(f"  Test:  X={X_test.shape}, y={y_test.shape}", end="")
        if self.include_power_feature:
            print(f", power={power_test.shape}")
        else:
            print()

        return (X_train, y_train, power_train), (X_val, y_val, power_val), (X_test, y_test, power_test)

    def train(self, df_combined, epochs=100, batch_size=16,
              split_strategy='trial', test_trials=[], val_trials=[],
              verbose=1):
        """
        Train the CNN-LSTM model

        Parameters:
        - df_combined: Combined dataframe with all trial data
        - epochs: Number of training epochs
        - batch_size: Training batch size
        - split_strategy: Only 'trial' supported in this integrated version
        - test_trials, val_trials: Lists of trial names for splits
        - verbose: Training verbosity
        """
        if split_strategy != 'trial':
            raise ValueError("Only 'trial' split strategy is supported in this integrated version")

        # Prepare data with trial-based splitting
        (X_train, y_train, power_train), (X_val, y_val, power_val), (X_test, y_test, power_test) = self.prepare_data_by_trial(
            df_combined,
            [t for t in df_combined['trial'].unique() if t not in test_trials + val_trials],
            val_trials,
            test_trials
        )

        if len(X_train) == 0:
            raise ValueError("No training data generated. Check your data and parameters.")

        # Store test data for evaluation
        self.X_test = X_test
        self.y_test = y_test
        self.power_test = power_test

        # Build model
        print("Building CNN-LSTM model...")
        self.model = self.build_model()
        print("Model architecture:")
        self.model.summary()

        # Define callbacks
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=20,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=15,
                min_lr=1e-6,
                verbose=1
            ),
            ModelCheckpoint(
                'best_cnn_lstm_model.h5',
                monitor='val_loss',
                save_best_only=True,
                verbose=1
            )
        ]

        # Prepare training inputs
        if self.include_power_feature:
            train_inputs = [X_train, power_train]
            val_inputs = [X_val, power_val] if len(X_val) > 0 else None
        else:
            train_inputs = X_train
            val_inputs = X_val if len(X_val) > 0 else None

        # Train the model
        print("Training model...")
        validation_data = (val_inputs, y_val) if val_inputs is not None and len(X_val) > 0 else None

        self.history = self.model.fit(
            train_inputs, y_train,
            validation_data=validation_data,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=verbose
        )

        return self.history

    def evaluate_model(self):
        """Evaluate model performance on test set"""
        if self.X_test is None or self.y_test is None or len(self.X_test) == 0:
            raise ValueError("No test data available. Train the model first.")

        # Make predictions
        if self.include_power_feature:
            test_inputs = [self.X_test, self.power_test]
        else:
            test_inputs = self.X_test

        y_pred = self.model.predict(test_inputs, verbose=0)

        # Calculate metrics for each feature
        results = {}
        for i, feature in enumerate(self.target_features):
            y_true_flat = self.y_test[:, :, i].flatten()
            y_pred_flat = y_pred[:, :, i].flatten()

            results[feature] = {
                'mse': mean_squared_error(y_true_flat, y_pred_flat),
                'rmse': np.sqrt(mean_squared_error(y_true_flat, y_pred_flat)),
                'mae': mean_absolute_error(y_true_flat, y_pred_flat),
                'r2': r2_score(y_true_flat, y_pred_flat)
            }

        return results, y_pred

    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            raise ValueError("No training history available.")

        fig, axes = plt.subplots(1, 2, figsize=(15, 5))

        # Plot loss
        axes[0].plot(self.history.history['loss'], label='Training Loss')
        if 'val_loss' in self.history.history:
            axes[0].plot(self.history.history['val_loss'], label='Validation Loss')
        axes[0].set_title('Model Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].legend()
        axes[0].grid(True)

        # Plot MAE
        axes[1].plot(self.history.history['mae'], label='Training MAE')
        if 'val_mae' in self.history.history:
            axes[1].plot(self.history.history['val_mae'], label='Validation MAE')
        axes[1].set_title('Model MAE')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('MAE')
        axes[1].legend()
        axes[1].grid(True)

        plt.tight_layout()
        plt.show()

    def plot_predictions(self, n_samples=3):
        """Plot sample predictions vs actual values"""
        if self.X_test is None or len(self.X_test) == 0:
            raise ValueError("No test data available.")

        n_samples = min(n_samples, len(self.X_test))

        if self.include_power_feature:
            test_inputs = [self.X_test[:n_samples], self.power_test[:n_samples]]
        else:
            test_inputs = self.X_test[:n_samples]

        y_pred = self.model.predict(test_inputs, verbose=0)

        # Inverse transform to original scale
        y_true_reshaped = self.y_test[:n_samples].reshape(-1, self.n_features)
        y_pred_reshaped = y_pred.reshape(-1, self.n_features)

        y_true_orig = self.target_scaler.inverse_transform(y_true_reshaped)
        y_pred_orig = self.target_scaler.inverse_transform(y_pred_reshaped)

        # Reshape back
        y_true_orig = y_true_orig.reshape(n_samples, self.forecast_horizon, self.n_features)
        y_pred_orig = y_pred_orig.reshape(n_samples, self.forecast_horizon, self.n_features)

        # Plot for each feature
        fig, axes = plt.subplots(len(self.target_features), 1,
                               figsize=(15, 3 * len(self.target_features)))
        if len(self.target_features) == 1:
            axes = [axes]

        for i, feature in enumerate(self.target_features):
            for j in range(n_samples):
                label_suffix = ""
                if self.include_power_feature:
                    power_orig = self.power_scaler.inverse_transform([[self.power_test[j]]])[0][0]
                    label_suffix = f" (P={power_orig:.0f}W)"

                axes[i].plot(range(self.forecast_horizon), y_true_orig[j, :, i],
                           'o-', label=f'Actual (Sample {j+1}{label_suffix})', alpha=0.7)
                axes[i].plot(range(self.forecast_horizon), y_pred_orig[j, :, i],
                           's--', label=f'Predicted (Sample {j+1}{label_suffix})', alpha=0.7)

            axes[i].set_title(f'{feature} - Predictions vs Actual')
            axes[i].set_xlabel('Forecast Step')
            axes[i].set_ylabel('Value')
            axes[i].legend()
            axes[i].grid(True)

        plt.tight_layout()
        plt.show()


def load_data_from_folders(image_base_path, label_base_path):
    """
    Load data where each CSV corresponds to a folder of images

    Expected structure:
    - images/
      - 017_Al6061_P200W_t4ms_U18G16/
        - 017_Al6061_P200W_t4ms_U18G16_spot0255_original.jpg
        - 017_Al6061_P200W_t4ms_U18G16_spot0256_original.jpg
        - ...
      - 017_Al6061_P250W_t4ms_U18G16/
        - 017_Al6061_P250W_t4ms_U18G16_spot0255_original.jpg
        - ...
    - labels/
      - AM_label_017_Al6061_P200W_t4ms_U18G16.csv
      - AM_label_017_Al6061_P250W_t4ms_U18G16.csv
      - ...
    """
    all_data = []

    # Get all CSV files
    if not os.path.exists(label_base_path):
        raise ValueError(f"Label directory not found: {label_base_path}")

    csv_files = [f for f in os.listdir(label_base_path) if f.endswith('.csv')]
    print(f"Found {len(csv_files)} CSV files: {csv_files}")

    for csv_file in csv_files:
        # Extract trial name from CSV filename
        # Remove 'AM_label_' prefix and '.csv' suffix
        trial_name = csv_file.replace('.csv', '').replace('AM_label_', '')

        # Load CSV
        csv_path = os.path.join(label_base_path, csv_file)
        df = pd.read_csv(csv_path)
        print(f"\nLoading {csv_file}: {df.shape}")
        print(f"Columns: {list(df.columns)}")

        # Create image folder path
        image_folder = os.path.join(image_base_path, trial_name)

        if os.path.exists(image_folder):
            # Get available image files
            image_files = sorted([f for f in os.listdir(image_folder)
                                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff'))])
            print(f"Found {len(image_files)} images in {trial_name}")

            # Parse first image to understand the naming pattern
            image_path_template = None
            if image_files:
                sample_image_filename = image_files[0]
                print(f"Sample image filename: {sample_image_filename}")
                # Try to extract the full filename structure before the spot index
                match = re.match(r'(.*_spot)\d+_original\.jpg', sample_image_filename)
                if match:
                    prefix = match.group(1)
                    image_path_template = os.path.join(image_folder, f"{prefix}{{x:04d}}_original.jpg")
                    print(f"Inferred image path template: {image_path_template}")
                else:
                    print(f"Could not infer image path template from {sample_image_filename}")
                    # Fallback to the previous method if template inference fails
                    if 'spot_index' in df.columns:
                        image_path_template = os.path.join(image_folder, f"{trial_name}_spot{{x:04d}}_original.jpg")
                        print(f"Using fallback image path template based on spot_index: {image_path_template}")
                    elif 'abs_index' in df.columns:
                        image_path_template = os.path.join(image_folder, f"{trial_name}_spot{{x:04d}}_original.jpg")
                        print(f"Using fallback image path template based on abs_index: {image_path_template}")


            if image_path_template and ('spot_index' in df.columns or 'abs_index' in df.columns):
                # Use the inferred or fallback template with available index
                index_col = 'spot_index' if 'spot_index' in df.columns else 'abs_index'
                df['image_path'] = df[index_col].apply(lambda x: image_path_template.format(x=x))
                print(f"Generated image paths using column '{index_col}' and template.")

                # Verify if generated paths exist for the first few rows
                existing_paths_check = df['image_path'].head().apply(os.path.exists)
                if not existing_paths_check.all():
                    print(f"Warning: Generated image paths using template do not exist for {trial_name}")
                    for path in df['image_path'].head()[~existing_paths_check]:
                         print(f"  Generated but missing: {path}")
                    # If generated paths don't exist, try sequential matching as a last resort
                    if len(image_files) == len(df):
                        df['image_path'] = [os.path.join(image_folder, f) for f in image_files]
                        print("Falling back to sequential image matching")
                    else:
                        print(f"Warning: Sequential matching not possible (Mismatch between CSV rows ({len(df)}) and images ({len(image_files)}) in {trial_name})")
                        continue # Skip trial if no reliable method works

            # Method 3: Sequential matching if no index columns or template failed
            elif len(image_files) == len(df):
                df['image_path'] = [os.path.join(image_folder, f) for f in image_files]
                print("Using sequential image matching")
            else:
                print(f"Warning: Mismatch between CSV rows ({len(df)}) and images ({len(image_files)}) and no index/template method worked for {trial_name}")
                continue # Skip trial if no reliable method works


            # Extract additional features from image filenames (using a sample filename)
            if image_files: # Ensure there's at least one image to sample from
                 forecaster_temp = CNNLSTMKeyholeForecaster() # Re-initialize for safe use of methods
                 df['laser_power'] = df['image_path'].apply(
                     lambda x: forecaster_temp.extract_power_from_filename(os.path.basename(x))
                 )
                 df['spot_index'] = df['image_path'].apply(
                     lambda x: forecaster_temp.extract_spot_index_from_filename(os.path.basename(x))
                 )
            else:
                 print(f"Warning: No images found in {image_folder}, cannot extract features.")
                 continue # Skip trial if no images


            # Verify some paths exist
            existing_paths = df['image_path'].head(5).apply(os.path.exists)
            if not existing_paths.all():
                print(f"Warning: Some image paths don't exist in {trial_name} even after generation attempts.")
                missing_paths = df['image_path'].head(5)[~existing_paths]
                for path in missing_paths:
                    print(f"  Missing: {path}")
                continue # Skip trial if image paths are still missing

            else:
                print(f"✓ Verified image paths exist")

            # Print power distribution for this trial
            unique_powers = df['laser_power'].unique()
            print(f"Laser powers in this trial: {unique_powers}")

            # Add trial identifier
            df['trial'] = trial_name
            all_data.append(df)
            print(f"✓ Loaded {len(df)} samples from {trial_name}")

        else:
            print(f"✗ Warning: Image folder {image_folder} not found for {csv_file}")

    if not all_data:
        raise ValueError("No data loaded! Check your folder structure and paths.")

    # Combine all data
    combined_df = pd.concat(all_data, ignore_index=True)

    # Convert target features and laser power to numeric, coercing errors
    forecaster_temp = CNNLSTMKeyholeForecaster() # Re-initialize for safe use
    features_to_convert = list(forecaster_temp.target_features) + ['laser_power']
    for feature in features_to_convert:
        if feature in combined_df.columns:
             combined_df[feature] = pd.to_numeric(combined_df[feature], errors='coerce')


    print(f"\n" + "="*50)
    print(f"COMBINED DATASET")
    print(f"="*50)
    print(f"Total samples: {len(combined_df)}")
    print(f"Trials: {sorted(combined_df['trial'].unique())}")
    print(f"Laser power range: {combined_df['laser_power'].min()}W - {combined_df['laser_power'].max()}W")
    print(f"Unique laser powers: {sorted(combined_df['laser_power'].unique())}")
    if not combined_df.empty:
        print(f"Sample image path: {combined_df['image_path'].iloc[0]}")
        print(f"Image exists: {os.path.exists(combined_df['image_path'].iloc[0])}")
    else:
         print("Combined dataframe is empty.")


    return combined_df


def define_trial_splits(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_seed=42):
    """Split trials into train/validation/test sets ensuring power distribution"""
    trials = sorted(df['trial'].unique())
    n_trials = len(trials)

    if n_trials == 0:
        print("Warning: No trials available for splitting.")
        return [], [], []

    # Get power information for each trial
    trial_powers = {}
    for trial in trials:
        trial_data = df[df['trial'] == trial]
        if not trial_data.empty and 'laser_power' in trial_data.columns:
             trial_powers[trial] = trial_data['laser_power'].iloc[0]  # Assuming consistent power per trial
        else:
             print(f"Warning: Cannot get power for trial {trial}, skipping.")
             continue # Skip trials with no data or missing power

    # Filter out trials that couldn't get power information
    valid_trials = list(trial_powers.keys())
    n_valid_trials = len(valid_trials)

    if n_valid_trials < 3: # Need at least 3 trials for train/val/test split
         print(f"Warning: Only {n_valid_trials} valid trials found. Cannot perform train/val/test split.")
         return valid_trials, [], [] # Return all valid trials as training

    print(f"\nTrial power distribution (from {n_valid_trials} valid trials):")
    for trial, power in trial_powers.items():
        print(f"  {trial}: {power}W")


    # Calculate split indices (adjust based on valid trials)
    n_train = int(n_valid_trials * train_ratio)
    n_val = int(n_valid_trials * val_ratio)

    # Ensure at least one trial in train and test if possible
    if n_train == 0 and n_valid_trials > 0: n_train = 1
    if n_valid_trials - n_train - n_val == 0 and n_valid_trials > n_train: n_val = 1 # Ensure at least one val if possible

    # Stratified split to ensure power distribution is maintained across splits
    import random
    random.seed(random_seed)

    # Group trials by power (using only valid trials)
    power_groups = {}
    for trial in valid_trials:
        power = trial_powers[trial]
        if power not in power_groups:
            power_groups[power] = []
        power_groups[power].append(trial)

    # Shuffle trials within each power group
    for power in power_groups:
        random.shuffle(power_groups[power])

    # Distribute trials from each power group across splits
    train_trials, val_trials, test_trials = [], [], []
    all_valid_trials_shuffled = []
    for power in sorted(power_groups.keys()):
         all_valid_trials_shuffled.extend(power_groups[power])


    # Simple sequential split from shuffled list if stratified is complex with small numbers
    train_trials = all_valid_trials_shuffled[:n_train]
    val_trials = all_valid_trials_shuffled[n_train:n_train + n_val]
    test_trials = all_valid_trials_shuffled[n_train + n_val:]


    print(f"\nStratified trial splits (seed={random_seed}):")
    print(f"  Train trials ({len(train_trials)}): {train_trials}")
    print(f"  Val trials ({len(val_trials)}): {val_trials}")
    print(f"  Test trials ({len(test_trials)}): {test_trials}")

    # Show data and power distribution
    for split_name, trial_list in [('Train', train_trials), ('Val', val_trials), ('Test', test_trials)]:
        split_data = df[df['trial'].isin(trial_list)]
        if not split_data.empty:
            powers_in_split = sorted(split_data['laser_power'].unique())
            print(f"  {split_name}: {len(split_data)} samples, Powers: {powers_in_split}W")
        else:
            print(f"  {split_name}: 0 samples")


    return train_trials, val_trials, test_trials


def analyze_power_vs_performance(forecaster, df_combined):
    """
    Analyze how model performance varies with laser power
    """
    if forecaster.X_test is None or not forecaster.include_power_feature or len(forecaster.X_test) == 0:
        print("No test data or power feature not included, or test data is empty.")
        return

    # Make predictions
    test_inputs = [forecaster.X_test, forecaster.power_test]
    y_pred = forecaster.model.predict(test_inputs, verbose=0)

    # Get original power values
    # Ensure power_test is not empty
    if forecaster.power_test is not None and len(forecaster.power_test) > 0:
        power_orig = forecaster.power_scaler.inverse_transform(forecaster.power_test.reshape(-1, 1)).flatten()
    else:
        print("No power data available for power vs performance analysis.")
        return


    # Calculate metrics by power level
    unique_powers = np.unique(power_orig)
    power_metrics = {}

    for power in unique_powers:
        power_mask = power_orig == power
        if np.sum(power_mask) == 0:
            continue

        power_metrics[power] = {}

        for i, feature in enumerate(forecaster.target_features):
            y_true_power = forecaster.y_test[power_mask, :, i].flatten()
            y_pred_power = y_pred[power_mask, :, i].flatten()

            # Ensure there are samples for this power level and feature
            if len(y_true_power) > 0:
                power_metrics[power][feature] = {
                    'rmse': np.sqrt(mean_squared_error(y_true_power, y_pred_power)),
                    'mae': mean_absolute_error(y_true_power, y_pred_power),
                    'r2': r2_score(y_true_power, y_pred_power),
                    'n_samples': len(y_true_power)
                }

    # Print results
    print("\nPerformance by Laser Power:")
    print("=" * 60)
    if not power_metrics:
        print("No performance data available by power level.")
        return power_metrics

    for power in sorted(power_metrics.keys()):
        print(f"\nPower: {power:.0f}W")
        for feature in forecaster.target_features:
            if feature in power_metrics[power]:
                metrics = power_metrics[power][feature]
                print(f"  {feature}:")
                print(f"    RMSE: {metrics['rmse']:.4f}")
                print(f"    MAE:  {metrics['mae']:.4f}")
                print(f"    R²:   {metrics['r2']:.4f}")
                print(f"    Samples: {metrics['n_samples']}")
            else:
                 print(f"  {feature}: No data for this power level.")


    return power_metrics


def plot_power_analysis(forecaster, df_combined):
    """
    Create visualizations of power vs model performance
    """
    if not forecaster.include_power_feature:
        print("Power feature not included in model")
        return

    power_metrics = analyze_power_vs_performance(forecaster, df_combined)
    if not power_metrics:
        print("No power metrics available to plot.")
        return

    powers = sorted(power_metrics.keys())
    n_features = len(forecaster.target_features)

    # Determine how many subplots are needed for metrics + distribution
    num_metric_plots = min(len(['rmse', 'mae', 'r2']), n_features)
    num_plots = num_metric_plots + 1 # metrics + distribution
    n_cols = 3 # Max columns
    n_rows = (num_plots + n_cols - 1) // n_cols # Calculate rows needed

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten()

    metrics_to_plot = ['rmse', 'mae', 'r2']
    current_plot_idx = 0

    for i, metric in enumerate(metrics_to_plot):
        if current_plot_idx >= len(axes) -1: # Leave space for distribution plot
             break

        # Check if metric data is available for at least one power/feature combination
        has_metric_data = False
        for power in powers:
            for feature in forecaster.target_features:
                if feature in power_metrics[power] and metric in power_metrics[power][feature]:
                    has_metric_data = True
                    break
            if has_metric_data: break

        if has_metric_data:
            ax = axes[current_plot_idx]
            current_plot_idx += 1

            for j, feature in enumerate(forecaster.target_features):
                values = [power_metrics[power].get(feature, {}).get(metric, None) for power in powers]
                # Filter out None values for plotting
                plottable_values = [v for v in values if v is not None]
                plottable_powers = [p for p, v in zip(powers, values) if v is not None]

                if plottable_values:
                    ax.plot(plottable_powers, plottable_values, 'o-', label=feature, alpha=0.7)


            ax.set_xlabel('Laser Power (W)')
            ax.set_ylabel(metric.upper())
            ax.set_title(f'{metric.upper()} vs Laser Power')
            ax.legend()
            ax.grid(True)
        else:
            print(f"No data available to plot {metric.upper()}.")


    # Power distribution plot (always last)
    if current_plot_idx < len(axes):
        ax = axes[current_plot_idx]
        current_plot_idx += 1

        if not df_combined.empty and 'laser_power' in df_combined.columns:
            power_counts = df_combined['laser_power'].value_counts().sort_index()
            if not power_counts.empty:
                ax.bar(power_counts.index, power_counts.values, alpha=0.7)
                ax.set_xlabel('Laser Power (W)')
                ax.set_ylabel('Number of Samples')
                ax.set_title('Data Distribution by Laser Power')
                ax.set_xticks(power_counts.index) # Ensure all power levels are shown
                ax.grid(True)
            else:
                 ax.set_title('No data available for Power Distribution')
        else:
            ax.set_title('No data available for Power Distribution')


    # Hide any unused subplots
    for i in range(current_plot_idx, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()


# ====================================================================
# MAIN EXECUTION SCRIPT
# ====================================================================

if __name__ == "__main__":
    print("="*60)
    print("CNN-LSTM KEYHOLE FORECASTER WITH LASER POWER")
    print("="*60)

    # 1. Mount Google Drive
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

    # 2. CONFIGURE YOUR PATHS
    # UPDATE THESE PATHS TO MATCH YOUR GOOGLE DRIVE STRUCTURE
    BASE_PATH = '/content/drive/MyDrive/label_keyhole_porosity'
    IMAGE_BASE_PATH = f'{BASE_PATH}/AM_image'  # Folder containing image subfolders
    LABEL_BASE_PATH = f'{BASE_PATH}/AM_label'  # Folder containing CSV files

    print(f"Image base path: {IMAGE_BASE_PATH}")
    print(f"Label base path: {LABEL_BASE_PATH}")

    # 3. Load and combine all data
    try:
        print(f"\nLoading data from folders...")
        df_combined = load_data_from_folders(IMAGE_BASE_PATH, LABEL_BASE_PATH)
        if df_combined.empty:
            print("ERROR: Data loading resulted in an empty dataset. Exiting.")
            exit(1)

    except Exception as e:
        print(f"ERROR loading data: {e}")
        print("\nPlease check:")
        print("1. Your Google Drive paths are correct")
        print("2. The folder structure matches the expected format")
        print("3. CSV files and image folders have matching names")
        print("4. Image files follow the naming convention: XXX_Al6061_PXXXW_t4ms_U18G16_spotXXXX_original.jpg")
        exit(1)

    # 4. Configure target features
    print(f"\nAvailable columns: {list(df_combined.columns)}")

    # UPDATE THESE TO MATCH YOUR ACTUAL CSV COLUMNS
    target_features = [
        'v2_keyhole_depth/pixel',
        'v2_keyhole_width/pixel',
        'v2_keyhole_depth/um',
        'v2_keyhole_width/um',
        'porosity'
    ]

    # Verify target features exist
    available_features = [f for f in target_features if f in df_combined.columns]
    missing_features = [f for f in target_features if f not in df_combined.columns]

    if missing_features:
        print(f"Warning: Missing features: {missing_features}")
        print("Available numeric columns:")
        numeric_cols = df_combined.select_dtypes(include=[np.number]).columns.tolist()
        print(numeric_cols)
        target_features = available_features

    if not target_features:
        print("ERROR: No valid target features found!")
        exit(1)

    print(f"Using target features: {target_features}")
    print(f"\nTarget feature statistics:")
    print(df_combined[target_features].describe())

    # 5. Split trials with power stratification
    train_trials, val_trials, test_trials = define_trial_splits(
        df_combined,
        train_ratio=0.7,
        val_ratio=0.15,
        test_ratio=0.15,
        random_seed=42
    )

    if len(train_trials) == 0:
        print("ERROR: No training trials assigned after splitting. Cannot proceed with training. Check your data and split ratios.")
        exit(1)

    # 6. Initialize forecaster with power feature
    forecaster = CNNLSTMKeyholeForecaster(
        sequence_length=20,      # Number of frames to look back
        forecast_horizon=5,      # Number of steps to predict ahead
        img_height=128,         # Resize images to this height
        img_width=128,          # Resize images to this width
        channels=1,             # Grayscale images
        target_features=target_features,
        include_power_feature=True  # Include laser power as input feature
    )

    # 7. Fit scalers on training data only
    train_data = df_combined[df_combined['trial'].isin(train_trials)].copy() # Use copy to avoid SettingWithCopyWarning

    # Check if training data is empty before fitting scalers
    if len(train_data) == 0:
        raise ValueError("Training data is empty after splitting trials. Cannot fit scalers.")

    # Drop rows with NaN in target features or laser power before fitting scalers
    cols_to_check_for_nan = list(target_features) + ['laser_power']
    train_data.dropna(subset=cols_to_check_for_nan, inplace=True)

    if len(train_data) == 0:
        raise ValueError("Training data is empty after dropping rows with missing target features or laser power. Cannot fit scalers.")

    forecaster.target_scaler.fit(train_data[target_features])
    forecaster.power_scaler.fit(train_data[['laser_power']])
    print(f"\nFitted scalers on {len(train_data)} training samples (after dropping NaNs)")
    print(f"Power range in training data: {train_data['laser_power'].min()}W - {train_data['laser_power'].max()}W")

    # 8. Train the model
    print(f"\n" + "="*50)
    print("STARTING TRAINING WITH LASER POWER FEATURE")
    print("="*50)
    print("This process may take 30-60 minutes...")
    print("The model will learn to predict keyhole behavior based on both")
    print("image sequences AND laser power levels")

    try:
        history = forecaster.train(
            df_combined,                # Combined dataframe
            epochs=50,                  # Number of training epochs
            batch_size=8,              # Batch size (adjust based on GPU memory)
            split_strategy='trial',     # Use trial-based splitting
            test_trials=test_trials,    # Trials for testing
            val_trials=val_trials      # Trials for validation
        )

        print("✓ Training completed successfully!")

        # 9. Evaluate the model
        print(f"\n" + "="*50)
        print("MODEL EVALUATION")
        print("="*50)

        if forecaster.X_test is not None and len(forecaster.X_test) > 0:
            results, predictions = forecaster.evaluate_model()

            print("Overall Model Performance:")
            print("-" * 40)
            for feature, metrics in results.items():
                print(f"{feature}:")
                print(f"  RMSE: {metrics['rmse']:.4f}")
                print(f"  MAE:  {metrics['mae']:.4f}")
                print(f"  R²:   {metrics['r2']:.4f}")
                print()

            # 10. Power-specific analysis
            print("Analyzing performance by laser power...")
            analyze_power_vs_performance(forecaster, df_combined)

            # 11. Plot results
            print("Generating plots...")
            forecaster.plot_training_history()
            forecaster.plot_predictions(n_samples=3)
            plot_power_analysis(forecaster, df_combined)

        else:
            print("No test data available for evaluation")

    except Exception as e:
        print(f"Training failed: {e}")
        print("\nCommon solutions:")
        print("1. Reduce batch_size (try 4 or 2) if you get memory errors")
        print("2. Reduce sequence_length if you have insufficient data")
        print("3. Check that image paths are correct and images load properly")
        print("4. Ensure each trial has enough samples for sequence generation")
        print("5. Verify target features exist and contain valid numeric data")
        print("6. Check that laser power values are extracted correctly from filenames")
        print("7. Ensure there are enough valid trials for train/val/test splits.")


        # Print debug information
        print(f"\nDebug Info:")
        if 'df_combined' in locals():
            print(f"  Total samples in combined_df: {len(df_combined)}")
            print(f"  Trials in combined_df: {len(df_combined['trial'].unique()) if 'trial' in df_combined.columns else 0}")
            if 'train_data' in locals():
                print(f"  Training samples (after NaN drop): {len(train_data)}")
            if 'forecaster' in locals() and hasattr(forecaster, 'sequence_length'):
                 print(f"  Required samples per trial for sequence generation: {forecaster.sequence_length + forecaster.forecast_horizon - 1}")
            if 'train_trials' in locals():
                 print(f"  Train/Val/Test trials (after filtering): {len(train_trials)}/{len(val_trials)}/{len(test_trials)}")
            if not df_combined.empty and 'laser_power' in df_combined.columns:
                 print(f"  Power range in combined_df: {df_combined['laser_power'].min()}W - {df_combined['laser_power'].max()}W")
        else:
            print("  df_combined is not defined. Data loading failed early.")


    print(f"\n" + "="*60)
    print("SETUP SUMMARY")
    print("="*60)
    if 'df_combined' in locals():
        print(f"Dataset: {len(df_combined)} samples across {len(df_combined['trial'].unique()) if 'trial' in df_combined.columns else 0} trials")
        if 'forecaster' in locals() and hasattr(forecaster, 'img_height'):
            print(f"Image size: {forecaster.img_height}x{forecaster.img_width}")
            print(f"Sequence length: {forecaster.sequence_length}")
            print(f"Forecast horizon: {forecaster.forecast_horizon}")
            print(f"Target features: {target_features}")
            print(f"Laser power feature: {'Enabled' if forecaster.include_power_feature else 'Disabled'}")
        if not df_combined.empty and 'laser_power' in df_combined.columns:
             print(f"Power range: {df_combined['laser_power'].min()}W - {df_combined['laser_power'].max()}W")
        if 'train_trials' in locals():
             print(f"Train/Val/Test trials: {len(train_trials)}/{len(val_trials)}/{len(test_trials)}")
    else:
        print("Data loading failed.")


    if 'history' in locals():
        print(f"Training epochs completed: {len(history.history['loss'])}")
        print(f"Final training loss: {history.history['loss'][-1]:.4f}")
        if 'val_loss' in history.history:
            print(f"Final validation loss: {history.history['val_loss'][-1]:.4f}")

    print("\n" + "="*60)
    print("NEXT STEPS")
    print("="*60)
    print("1. The model now considers laser power in its predictions")
    print("2. You can predict on new sequences with different power levels")
    print("3. Save your model: forecaster.model.save('cnn_lstm_with_power.h5')")
    print("4. Experiment with different power levels for process optimization")
    print("5. Use power-specific analysis to identify optimal operating conditions")

    if 'forecaster' in locals() and forecaster.model is not None:
         print("\nModel ready for power-aware predictions!")
    else:
         print("\nModel training failed. Please check debug information above.")


# ====================================================================
# UTILITY FUNCTIONS FOR POWER-AWARE PREDICTIONS
# ====================================================================

def predict_with_power(forecaster, image_sequence, power_level):
    """
    Make predictions for a given image sequence and power level

    Parameters:
    - forecaster: Trained CNN-LSTM forecaster
    - image_sequence: Sequence of images (sequence_length, height, width, channels)
    - power_level: Laser power in Watts

    Returns:
    - Predicted keyhole dimensions
    """
    if not forecaster.include_power_feature:
        print("Model was not trained with power feature")
        return None

    if len(image_sequence.shape) == 4:
        image_sequence = image_sequence.reshape(1, forecaster.sequence_length,
                                              forecaster.img_height, forecaster.img_width, forecaster.channels)

    # Normalize power
    power_normalized = forecaster.power_scaler.transform([[power_level]])[0]
    power_input = power_normalized.reshape(1, 1)

    # Make prediction
    prediction = forecaster.model.predict([image_sequence, power_input], verbose=0)

    # Inverse transform to original scale
    prediction_reshaped = prediction.reshape(-1, forecaster.n_features)
    prediction_orig = forecaster.target_scaler.inverse_transform(prediction_reshaped)

    return prediction_orig.reshape(forecaster.forecast_horizon, forecaster.n_features)


def power_optimization_study(forecaster, sample_image_sequence, power_range=(200, 450, 50)):
    """
    Study the effect of different power levels on predictions

    Parameters:
    - forecaster: Trained CNN-LSTM forecaster
    - sample_image_sequence: Sample image sequence for analysis
    - power_range: (min_power, max_power, step) in Watts

    Returns:
    - Dictionary with power levels and corresponding predictions
    """
    if not forecaster.include_power_feature:
        print("Model was not trained with power feature")
        return None

    powers = range(power_range[0], power_range[1] + 1, power_range[2])
    results = {}

    for power in powers:
        prediction = predict_with_power(forecaster, sample_image_sequence, power)
        results[power] = prediction

        print(f"Power {power}W predictions:")
        for i, feature in enumerate(forecaster.target_features):
            avg_pred = np.mean(prediction[:, i])
            print(f"  {feature}: {avg_pred:.4f}")
        print()

    return results


def plot_power_optimization(power_results, forecaster):
    """
    Plot the results of power optimization study
    """
    if not power_results:
        return

    powers = sorted(power_results.keys())
    n_features = len(forecaster.target_features)

    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()

    for i, feature in enumerate(forecaster.target_features):
        if i >= len(axes):
            break

        ax = axes[i]

        # Get average predictions for each power level
        avg_predictions = []
        for power in powers:
            avg_pred = np.mean(power_results[power][:, i])
            avg_predictions.append(avg_pred)

        ax.plot(powers, avg_predictions, 'o-', linewidth=2, markersize=8)
        ax.set_xlabel('Laser Power (W)')
        ax.set_ylabel(f'Predicted {feature}')
        ax.set_title(f'Effect of Laser Power on {feature}')
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

CNN-LSTM KEYHOLE FORECASTER WITH LASER POWER
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Image base path: /content/drive/MyDrive/label_keyhole_porosity/AM_image
Label base path: /content/drive/MyDrive/label_keyhole_porosity/AM_label

Loading data from folders...
Found 12 CSV files: ['AM_label_007.csv', 'AM_label_009.csv', 'AM_label_019.csv', 'AM_label_005.csv', 'AM_label_016.csv', 'AM_label_008.csv', 'AM_label_015.csv', 'AM_label_011.csv', 'AM_label_018.csv', 'AM_label_006.csv', 'AM_label_010.csv', 'AM_label_017.csv']

Loading AM_label_007.csv: (501, 7)
Columns: ['abs_index', 'rel_index', 'v2_keyhole_depth/pixel', 'v2_keyhole_width/pixel', 'v2_keyhole_depth/um', 'v2_keyhole_width/um', 'porosity']
Found 501 images in 007
Sample image filename: 007_Al6061_P250W_t4ms_U18G16_spot0255_original.jpg
Inferred image path template: /content/drive/MyDrive/label_keyhole_porosity/AM_image/0

In [ ]:
# ====================================================================
# UTILITY FUNCTIONS FOR POST-TRAINING ANALYSIS
# ====================================================================

def analyze_trial_predictions(forecaster, df_combined, trial_name, n_sequences=5):
    """
    Analyze predictions for a specific trial

    Parameters:
    - forecaster: Trained CNN-LSTM forecaster
    - df_combined: Combined dataframe
    - trial_name: Name of trial to analyze
    - n_sequences: Number of sequences to analyze
    """
    if forecaster.model is None:
        print("Model not trained yet!")
        return

    trial_data = df_combined[df_combined['trial'] == trial_name].sort_values('abs_index')
    print(f"Analyzing trial: {trial_name}")
    print(f"Trial data shape: {trial_data.shape}")

    # Load trial sequences
    trial_X, trial_y = forecaster.load_trial_sequences(trial_name, trial_data)

    if len(trial_X) == 0:
        print("No sequences generated for this trial")
        return

    # Make predictions
    n_sequences = min(n_sequences, len(trial_X))
    predictions = forecaster.model.predict(trial_X[:n_sequences], verbose=0)

    # Inverse transform
    y_true_reshaped = trial_y[:n_sequences].reshape(-1, forecaster.n_features)
    y_pred_reshaped = predictions.reshape(-1, forecaster.n_features)

    y_true_orig = forecaster.target_scaler.inverse_transform(y_true_reshaped)
    y_pred_orig = forecaster.target_scaler.inverse_transform(y_pred_reshaped)

    # Calculate metrics
    for i, feature in enumerate(forecaster.target_features):
        y_true_feat = y_true_orig[:, i]
        y_pred_feat = y_pred_orig[:, i]

        mse = mean_squared_error(y_true_feat, y_pred_feat)
        mae = mean_absolute_error(y_true_feat, y_pred_feat)
        r2 = r2_score(y_true_feat, y_pred_feat)

        print(f"{feature}: RMSE={np.sqrt(mse):.4f}, MAE={mae:.4f}, R²={r2:.4f}")

    return predictions, trial_X, trial_y


def save_model_and_scaler(forecaster, base_path="/content/drive/MyDrive/saved_models"):
    """
    Save the trained model and scaler

    Parameters:
    - forecaster: Trained CNN-LSTM forecaster
    - base_path: Path to save model files
    """
    import pickle
    import os

    if forecaster.model is None:
        print("No model to save!")
        return

    # Create directory if it doesn't exist
    os.makedirs(base_path, exist_ok=True)

    # Save model
    model_path = os.path.join(base_path, "cnn_lstm_keyhole_model.h5")
    forecaster.model.save(model_path)
    print(f"Model saved to: {model_path}")

    # Save scaler
    scaler_path = os.path.join(base_path, "target_scaler.pkl")
    with open(scaler_path, 'wb') as f:
        pickle.dump(forecaster.target_scaler, f)
    print(f"Scaler saved to: {scaler_path}")

    # Save configuration
    config = {
        'sequence_length': forecaster.sequence_length,
        'forecast_horizon': forecaster.forecast_horizon,
        'img_height': forecaster.img_height,
        'img_width': forecaster.img_width,
        'channels': forecaster.channels,
        'target_features': forecaster.target_features
    }

    config_path = os.path.join(base_path, "model_config.pkl")
    with open(config_path, 'wb') as f:
        pickle.dump(config, f)
    print(f"Configuration saved to: {config_path}")


def load_model_and_scaler(base_path="/content/drive/MyDrive/saved_models"):
    """
    Load a saved model and create forecaster

    Parameters:
    - base_path: Path where model files are saved

    Returns:
    - Loaded forecaster object
    """
    import pickle
    import tensorflow as tf

    # Load configuration
    config_path = os.path.join(base_path, "model_config.pkl")
    with open(config_path, 'rb') as f:
        config = pickle.load(f)

    # Create forecaster
    forecaster = CNNLSTMKeyholeForecaster(**config)

    # Load model
    model_path = os.path.join(base_path, "cnn_lstm_keyhole_model.h5")
    forecaster.model = tf.keras.models.load_model(model_path)

    # Load scaler
    scaler_path = os.path.join(base_path, "target_scaler.pkl")
    with open(scaler_path, 'rb') as f:
        forecaster.target_scaler = pickle.load(f)

    print(f"Model loaded from: {model_path}")
    print(f"Scaler loaded from: {scaler_path}")

    return forecaster


def predict_on_new_trial(forecaster, image_folder_path, start_sequence_length=None):
    """
    Make predictions on a new trial (folder of images)

    Parameters:
    - forecaster: Trained CNN-LSTM forecaster
    - image_folder_path: Path to folder containing images
    - start_sequence_length: Length of initial sequence (uses forecaster.sequence_length if None)

    Returns:
    - Array of predictions
    """
    if forecaster.model is None:
        print("Model not trained yet!")
        return None

    if start_sequence_length is None:
        start_sequence_length = forecaster.sequence_length

    # Get image files
    image_files = sorted([f for f in os.listdir(image_folder_path)
                         if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff'))])

    if len(image_files) < start_sequence_length:
        print(f"Need at least {start_sequence_length} images, found {len(image_files)}")
        return None

    # Load and preprocess images
    image_paths = [os.path.join(image_folder_path, f) for f in image_files]
    images = forecaster.load_and_preprocess_images(image_paths[:start_sequence_length])

    # Reshape for prediction
    input_sequence = images.reshape(1, start_sequence_length,
                                  forecaster.img_height, forecaster.img_width, forecaster.channels)

    # Make prediction
    prediction = forecaster.model.predict(input_sequence, verbose=0)

    # Inverse transform to original scale
    prediction_reshaped = prediction.reshape(-1, forecaster.n_features)
    prediction_orig = forecaster.target_scaler.inverse_transform(prediction_reshaped)

    print(f"Prediction for {len(image_files)} images:")
    for i, feature in enumerate(forecaster.target_features):
        print(f"{feature}: {prediction_orig[:, i]}")

    return prediction_orig


# ====================================================================
# EXAMPLE USAGE AFTER TRAINING
# ====================================================================

def example_post_training_analysis():
    """
    Example of how to use the forecaster after training
    """
    print("\n" + "="*60)
    print("EXAMPLE POST-TRAINING ANALYSIS")
    print("="*60)

    # This assumes you have a trained forecaster and data loaded
    # Uncomment and modify as needed:

    """
    # 1. Analyze specific trial
    analyze_trial_predictions(forecaster, df_combined, 'trial_001', n_sequences=3)

    # 2. Save model for future use
    save_model_and_scaler(forecaster, "/content/drive/MyDrive/my_models")

    # 3. Load saved model (in a new session)
    # loaded_forecaster = load_model_and_scaler("/content/drive/MyDrive/my_models")

    # 4. Make predictions on new trial
    # new_predictions = predict_on_new_trial(
    #     forecaster,
    #     "/content/drive/MyDrive/new_trial_images"
    # )
    """

    print("See function definitions above for detailed usage examples")
    print("Modify the paths and parameters according to your needs")